# Laboratorio 2
### Security Data Science
- Christian Pérez
- Andrei Portales

#### Preprocesamiento

In [1]:
import pandas as pd
import numpy as np
import re
from collections import Counter
from sklearn import feature_extraction, tree, model_selection, metrics
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
%matplotlib inline
from yellowbrick.features import Rank2D
from yellowbrick.features import RadViz

In [2]:
df1 = pd.read_csv('completeSpamAssassin.csv')
df1.head()

,Unnamed: 0,Body,Label
0,0,\nSave up to 70% on Life Insurance.\nWhy Spend...,1
1,1,1) Fight The Risk of Cancer!\nhttp://www.adcli...,1
2,2,1) Fight The Risk of Cancer!\nhttp://www.adcli...,1
3,3,##############################################...,1
4,4,I thought you might like these:\n1) Slim Down ...,1


In [3]:
df2 = pd.read_csv('enronSpamSubset.csv')
df2.head()

,Unnamed: 0.1,Unnamed: 0,Body,Label
0,2469,2469,Subject: stock promo mover : cwtd\n * * * urge...,1
1,5063,5063,Subject: are you listed in major search engine...,1
2,12564,12564,"Subject: important information thu , 30 jun 20...",1
3,2796,2796,Subject: = ? utf - 8 ? q ? bask your life with...,1
4,1468,1468,"Subject: "" bidstogo "" is places to go , things...",1


In [4]:
# delete unnamed:0.1 column and unnamed:0 column in df2
df2 = df2.drop(['Unnamed: 0.1', 'Unnamed: 0'], axis=1)
df2.head()

,Body,Label
0,Subject: stock promo mover : cwtd\n * * * urge...,1
1,Subject: are you listed in major search engine...,1
2,"Subject: important information thu , 30 jun 20...",1
3,Subject: = ? utf - 8 ? q ? bask your life with...,1
4,"Subject: "" bidstogo "" is places to go , things...",1


In [5]:
# delete unnamed:0 column in df1
df1 = df1.drop(['Unnamed: 0'], axis=1)
df1.head()

,Body,Label
0,\nSave up to 70% on Life Insurance.\nWhy Spend...,1
1,1) Fight The Risk of Cancer!\nhttp://www.adcli...,1
2,1) Fight The Risk of Cancer!\nhttp://www.adcli...,1
3,##############################################...,1
4,I thought you might like these:\n1) Slim Down ...,1


In [6]:
# merge df1 and df2
df = pd.concat([df1, df2], ignore_index=True)
df.head()

,Body,Label
0,\nSave up to 70% on Life Insurance.\nWhy Spend...,1
1,1) Fight The Risk of Cancer!\nhttp://www.adcli...,1
2,1) Fight The Risk of Cancer!\nhttp://www.adcli...,1
3,##############################################...,1
4,I thought you might like these:\n1) Slim Down ...,1


In [7]:
# size of the dataset
df.shape

(16046, 2)

In [8]:
import nltk
import unicodedata
import re
import contractions

In [9]:
# delete nan values
df = df.dropna()

In [10]:
# change to lowercase
df['Body'] = df['Body'].str.lower()
df.head()

,Body,Label
0,\nsave up to 70% on life insurance.\nwhy spend...,1
1,1) fight the risk of cancer!\nhttp://www.adcli...,1
2,1) fight the risk of cancer!\nhttp://www.adcli...,1
3,##############################################...,1
4,i thought you might like these:\n1) slim down ...,1


In [11]:
def removeAccents(text):
    text = unicodedata.normalize('NFKD', text).encode('ASCII', 'ignore').decode('utf-8', 'ignore')
    return text

# try remove accents
try:
    df['Body'] = df['Body'].apply(removeAccents)
except:
    pass
df.head()


,Body,Label
0,\nsave up to 70% on life insurance.\nwhy spend...,1
1,1) fight the risk of cancer!\nhttp://www.adcli...,1
2,1) fight the risk of cancer!\nhttp://www.adcli...,1
3,##############################################...,1
4,i thought you might like these:\n1) slim down ...,1


In [12]:
def removerCaracteresEspecialesNumerosSimbolos(texto, removerDigitos =False):
    patron = r'[^a-zA-Z0-9\s]' if not removerDigitos else r'[^a-zA-Z\s]'
    texto = re.sub(patron,'', texto)
    return texto

# try for every elemente remove special characters, numbers and symbols to the dataset

for i in range(len(df['Body'])):
    try:
        df['Body'][i] = removerCaracteresEspecialesNumerosSimbolos(df['Body'][i], True)
    except:
        pass

C:\Users\Christian\AppData\Local\Temp\ipykernel_29052\2470359608.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Body'][i] = removerCaracteresEspecialesNumerosSimbolos(df['Body'][i], True)


In [13]:
df.head()

,Body,Label
0,\nsave up to on life insurance\nwhy spend mor...,1
1,fight the risk of cancer\nhttpwwwadclickwspcf...,1
2,fight the risk of cancer\nhttpwwwadclickwspcf...,1
3,\n ...,1
4,i thought you might like these\n slim down gu...,1


In [14]:
#replace \n with space
df['Body'] = df['Body'].str.replace('\n', ' ')
df.head()

,Body,Label
0,save up to on life insurance why spend more ...,1
1,fight the risk of cancer httpwwwadclickwspcfm...,1
2,fight the risk of cancer httpwwwadclickwspcfm...,1
3,...,1
4,i thought you might like these slim down gua...,1


In [15]:
# expand contractions
def expandContractions(text):
    text = contractions.fix(text)
    return text

# try expand contractions
for i in range(len(df['Body'])):
    try:
        df['Body'][i] = expandContractions(df['Body'][i])
    except:
        pass

C:\Users\Christian\AppData\Local\Temp\ipykernel_29052\1862044096.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Body'][i] = expandContractions(df['Body'][i])


In [16]:
df.head()

,Body,Label
0,save up to on life insurance why spend more ...,1
1,fight the risk of cancer httpwwwadclickwspcfm...,1
2,fight the risk of cancer httpwwwadclickwspcfm...,1
3,...,1
4,i thought you might like these slim down gua...,1


In [17]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Christian\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [18]:
stopwords_english = nltk.corpus.stopwords.words('english')

In [19]:
def preprocesamiento(texto):
    texto = texto.split()
    texto = [palabra for palabra in texto if palabra not in stopwords_english]
    texto = ' '.join(texto)
    return texto

# try for every elemente remove stopwords to the dataset
for i in range(len(df['Body'])):
    try:
        df['Body'][i] = preprocesamiento(df['Body'][i])
    except:
        pass

df.head()

C:\Users\Christian\AppData\Local\Temp\ipykernel_29052\3097637006.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Body'][i] = preprocesamiento(df['Body'][i])


,Body,Label
0,save life insurance spend tolife quote savings...,1
1,fight risk cancer httpwwwadclickwspcfmospk sli...,1
2,fight risk cancer httpwwwadclickwspcfmospk sli...,1
3,adult club offers free membership instant acce...,1
4,thought might like slim guaranteed lose lbs da...,1


In [20]:
# delete rows with empty body
df = df[df['Body'] != '']
df.head()

,Body,Label
0,save life insurance spend tolife quote savings...,1
1,fight risk cancer httpwwwadclickwspcfmospk sli...,1
2,fight risk cancer httpwwwadclickwspcfmospk sli...,1
3,adult club offers free membership instant acce...,1
4,thought might like slim guaranteed lose lbs da...,1


In [21]:
# stemming
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()

def stemming(texto):
    texto = texto.split()
    texto = [stemmer.stem(palabra) for palabra in texto]
    texto = ' '.join(texto)
    return texto

# try for every elemente stem to the dataset
for i in range(len(df['Body'])):    
    try:
        df['Body'][i] = stemming(df['Body'][i])
    except:
        pass

df.head()

C:\Users\Christian\AppData\Local\Temp\ipykernel_29052\2707412909.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Body'][i] = stemming(df['Body'][i])


,Body,Label
0,save life insur spend tolif quot save ensur fa...,1
1,fight risk cancer httpwwwadclickwspcfmospk sli...,1
2,fight risk cancer httpwwwadclickwspcfmospk sli...,1
3,adult club offer free membership instant acces...,1
4,thought might like slim guarante lose lb day h...,1


In [22]:
# lemmatization
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

def lemmatization(texto):
    texto = texto.split()
    texto = [lemmatizer.lemmatize(palabra) for palabra in texto]
    texto = ' '.join(texto)
    return texto

# try for every elemente lemmatize to the dataset
for i in range(len(df['Body'])):
    try:
        df['Body'][i] = lemmatization(df['Body'][i])
    except:
        pass

df.head()

,Body,Label
0,save life insur spend tolif quot save ensur fa...,1
1,fight risk cancer httpwwwadclickwspcfmospk sli...,1
2,fight risk cancer httpwwwadclickwspcfmospk sli...,1
3,adult club offer free membership instant acces...,1
4,thought might like slim guarante lose lb day h...,1


In [37]:
# save the dataset
df.to_csv('dataset_clean.csv')
